# YOLO Diagrams 학습

다이어그램 객체 탐지를 위한 YOLO 학습

**원본 스크립트**: `train_yolov8_diagrams.sh`


In [ ]:
#!/usr/bin/env bash
set -euo pipefail


## ArchLens: AWS Diagram YOLOv8 최초 학습 스크립트


## - 원본: data/images, data/labels, data/classes.txt


## - 출력: data/yolo_diagrams/{images,labels}/{train,val}


## - 학습: yolo detect train ...


## 0. 기본 경로 설정


In [ ]:
PROJECT_ROOT="$HOME/workspace/hit-archlens-project"
DATA_ROOT="$PROJECT_ROOT/data"
RAW_IMG_DIR="$DATA_ROOT/images"
RAW_LBL_DIR="$DATA_ROOT/labels"
CLASS_FILE="$DATA_ROOT/classes.txt"
YOLO_ROOT="$DATA_ROOT/yolo_diagrams"
YOLO_IMG_TRAIN="$YOLO_ROOT/images/train"
YOLO_IMG_VAL="$YOLO_ROOT/images/val"
YOLO_LBL_TRAIN="$YOLO_ROOT/labels/train"
YOLO_LBL_VAL="$YOLO_ROOT/labels/val"
YOLO_DATA_YAML="$YOLO_ROOT/data.yaml"
TRAIN_RATIO=0.8        # train/val 비율
RANDOM_SEED=42         # 재현성 보장용
echo "[INFO] PROJECT_ROOT = $PROJECT_ROOT"
echo "[INFO] DATA_ROOT    = $DATA_ROOT"
echo "[INFO] RAW_IMG_DIR  = $RAW_IMG_DIR"
echo "[INFO] RAW_LBL_DIR  = $RAW_LBL_DIR"


## 1. 디렉터리 생성


In [ ]:
echo "[STEP 1] YOLO 데이터셋 디렉터리 생성..."
mkdir -p "$YOLO_IMG_TRAIN" "$YOLO_IMG_VAL"
mkdir -p "$YOLO_LBL_TRAIN" "$YOLO_LBL_VAL"


## 2. train/val 분할 + 복사


In [ ]:
echo "[STEP 2] train/val 분할 및 파일 복사..."


In [ ]:
import os
import shutil
import random
from pathlib import Path

PROJECT_ROOT = os.path.expanduser("~/workspace/hit-archlens-project")
DATA_ROOT    = os.path.join(PROJECT_ROOT, "data")

RAW_IMG_DIR  = os.path.join(DATA_ROOT, "images")
RAW_LBL_DIR  = os.path.join(DATA_ROOT, "labels")

YOLO_ROOT        = os.path.join(DATA_ROOT, "yolo_diagrams")
YOLO_IMG_TRAIN   = os.path.join(YOLO_ROOT, "images", "train")
YOLO_IMG_VAL     = os.path.join(YOLO_ROOT, "images", "val")
YOLO_LBL_TRAIN   = os.path.join(YOLO_ROOT, "labels", "train")
YOLO_LBL_VAL     = os.path.join(YOLO_ROOT, "labels", "val")

TRAIN_RATIO = 0.8
RANDOM_SEED = 42

random.seed(RANDOM_SEED)

# 1) 이미지-라벨 쌍 수집
img_dir = Path(RAW_IMG_DIR)
lbl_dir = Path(RAW_LBL_DIR)

pairs = []
for img_path in img_dir.iterdir():
    if not img_path.is_file():
        continue
    # 허용 확장자
    if img_path.suffix.lower() not in {".png", ".jpg", ".jpeg", ".webp"}:
        continue

    base = img_path.stem  # 확장자 제거 이름
    # 동일한 basename에 .txt 라벨이 있다고 가정
    lbl_path = lbl_dir / f"{base}.txt"
    if lbl_path.exists():
        pairs.append((img_path, lbl_path))

print(f"[PY] Found {len(pairs)} image-label pairs.")

if not pairs:
    raise SystemExit("[PY][ERROR] No image-label pairs found. 경로/확장자 확인 필요.")

# 2) 셔플 + train/val split
random.shuffle(pairs)
n_train = int(len(pairs) * TRAIN_RATIO)
train_pairs = pairs[:n_train]
val_pairs   = pairs[n_train:]

print(f"[PY] Train: {len(train_pairs)}  |  Val: {len(val_pairs)}")

# 3) 복사 함수
def copy_pair(pair_list, img_dst_dir, lbl_dst_dir):
    os.makedirs(img_dst_dir, exist_ok=True)
    os.makedirs(lbl_dst_dir, exist_ok=True)

    for img_path, lbl_path in pair_list:
        img_dst = os.path.join(img_dst_dir, img_path.name)
        lbl_dst = os.path.join(lbl_dst_dir, lbl_path.name)
        shutil.copy2(img_path, img_dst)
        shutil.copy2(lbl_path, lbl_dst)

copy_pair(train_pairs, YOLO_IMG_TRAIN, YOLO_LBL_TRAIN)
copy_pair(val_pairs,   YOLO_IMG_VAL,   YOLO_LBL_VAL)

print("[PY] 파일 복사 완료.")
PYCODE

########################
# 3. data.yaml 생성
########################
echo "[STEP 3] data.yaml 생성 (classes.txt 기반)..."

import os
from pathlib import Path
import textwrap

PROJECT_ROOT = os.path.expanduser("~/workspace/hit-archlens-project")
DATA_ROOT    = os.path.join(PROJECT_ROOT, "data")

YOLO_ROOT      = os.path.join(DATA_ROOT, "yolo_diagrams")
YOLO_DATA_YAML = os.path.join(YOLO_ROOT, "data.yaml")
CLASS_FILE     = os.path.join(DATA_ROOT, "classes.txt")

# 클래스 읽기
class_path = Path(CLASS_FILE)
if not class_path.exists():
    raise SystemExit(f"[PY][ERROR] classes.txt not found: {CLASS_FILE}")

with class_path.open("r", encoding="utf-8") as f:
    classes = [line.strip() for line in f if line.strip()]

print(f"[PY] Loaded {len(classes)} classes from classes.txt")

# data.yaml 작성
# YOLOv8에서 path는 data.yaml 위치 기준 상대 경로 사용 가능
yaml_lines = []
yaml_lines.append(f"path: {YOLO_ROOT}")
yaml_lines.append("train: images/train")
yaml_lines.append("val: images/val")
yaml_lines.append("")
yaml_lines.append("names:")

for idx, name in enumerate(classes):
    # 공백이 있는 경우에도 문제 없도록 그대로 문자열로 사용
    yaml_lines.append(f"  {idx}: {name}")

yaml_text = "\n".join(yaml_lines)

# 저장
with open(YOLO_DATA_YAML, "w", encoding="utf-8") as f:
    f.write(yaml_text)

print("[PY] data.yaml 생성 완료:")
print("------")
print(yaml_text)
print("------")
PYCODE

########################
# 4. YOLOv8 학습 실행
########################
echo "[STEP 4] YOLOv8 학습 시작..."
echo "[INFO] 현재 venv/환경에 ultralytics(yolo CLI)가 설치되어 있어야 합니다."

cd "$PROJECT_ROOT"

# 필요에 따라 model, epochs, imgsz, batch 조정
yolo detect train \
    data="$YOLO_DATA_YAML" \
    model=weights/yolov8m.pt \
    epochs=100 \
    imgsz=1024 \
    batch=8 \
    device=0 \
    name="aws_diagram_yolov8m_v1"

echo "[DONE] 학습 완료. runs/detect/aws_diagram_yolov8m_v1 디렉터리를 확인하세요."

